This project, which mines 28.5 years (!) worth of data from the television game show "Jeopardy!", is a Codecademy Pro Challenge Project. Codecademy provided a couple of lines of introductory code in a .py file, but I am choosing to complete the rest of the project as a Jupyter notebook. This will allow for easier, clearer commenting, and will hopefully make the results easier to see and interpret as well.

First let's start with the Codecademy provided code:

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

The original Codecademy setting for the second line of code above was "pd.set_option('display.max_colwidth', -1)", but this triggered a FutureWarning about passing a negative integer to set_option(). I followed the recommendation in the warning to pass "None" in instead of -1. This appeased the compiler.

Now, let's see what sort of data we are working with by importing the .csv file into a DataFrame and inspecting the DataFrame info():

In [2]:
jeopardy = pd.read_csv('jeopardy.csv')

print(jeopardy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Show Number  216930 non-null  int64 
 1    Air Date    216930 non-null  object
 2    Round       216930 non-null  object
 3    Category    216930 non-null  object
 4    Value       216930 non-null  object
 5    Question    216930 non-null  object
 6    Answer      216928 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.6+ MB
None


It is admittedly a little hard to see in this Jupyter notebook, but the columns "Air Date", "Round", "Category", "Value", "Question", and "Answer" all start with a leading whitespace. This is mildly to moderately annoying, so let's begin by renaming those columns to omit the whitespace. Oh, and let's turn the spaces between words in the first two columns into an underscore:

In [3]:
jeopardy.rename(columns = {"Show Number":"Show_Number", " Air Date":"Air_Date", " Round":"Round", \
                           " Category":"Category", " Value":"Value", " Question":"Question", " Answer":"Answer"},\
                           inplace = True)
print(jeopardy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Show_Number  216930 non-null  int64 
 1   Air_Date     216930 non-null  object
 2   Round        216930 non-null  object
 3   Category     216930 non-null  object
 4   Value        216930 non-null  object
 5   Question     216930 non-null  object
 6   Answer       216928 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.6+ MB
None


<font color = green>
    Codecademy: <br>
</font>
<font color = green>
We may want to eventually compute aggregate statistics, like .mean() on the " Value" column. But right now, the values in that column are strings. Convert the " Value" column to floats. If you’d like to, you can create a new column with the float values. <br>
</font>

Making a new column for the float values of the "Value" column sounds like a great idea, so I will do just that. I will go ahead and "write over" the previous Jeopardy DataFrame, as I have hard-coded its name in my function calls. <br>

All the numbers in the "Value" column have a leading $, so in order to convert that column to a float I will first need to get rid of that. I remember having to do this for a Codecademy exercise, so I will use what I did there for a model. But in doing so, we'll need to remember that there's no actual value associated with "Final Jeopardy" questions, which are of course included in this DataFrame. (I assume the same is true of "Daily Double" entries.) Lacking any better ideas, I'm going to give "Final Jeopardy" questions a value of 0 in the column "Value_Num", so that'll be something to keep in mind for the rest of the code. I will leave the actual "Value" of "Final Jeopardy" questions as "None," so at least that will be preserved and is yet another good reason to make the float values of the "Value" column its own separate column. I got bit in the butt trying to use .fillna() in the Value_Num column, because blank values are listed as the string "None" and not NaNs. Another call to .replace() took care of that.

In [4]:
jeopardy["Value_Num"] = jeopardy.Value.replace('[\$,]', '', regex=True)
jeopardy.Value_Num = jeopardy.Value_Num.replace('None', '0')
jeopardy.Value_Num = pd.to_numeric(jeopardy.Value_Num)

Let's see what the range of dates we're working with is:

In [5]:
print(min(jeopardy.Air_Date), max(jeopardy.Air_Date))

1984-09-10 2012-01-27


Actually, it'd be nice to have the Air_Date column separated into Year, Month, and Day to do this. I think I recall having to do something similar in a Codecademy exercise, so I'll use what I did there as a model.

In [6]:
jeopardy['Year'] = jeopardy.Air_Date.str[0:4]
jeopardy['Month'] = jeopardy.Air_Date.str[5:7]
jeopardy['Day'] = jeopardy.Air_Date.str[8:11]

jeopardy.Year = pd.to_numeric(jeopardy.Year)
jeopardy.Month = pd.to_numeric(jeopardy.Month)
jeopardy.Day = pd.to_numeric(jeopardy.Day)

With those columns renamed and extra columns cleaned/added, let's move on to the Codecademy prompted questions. I'll make them a different colour than my own comments (and clearly label them as coming from Codecademy).

<font color=green>
    Codecademy: <br> </font>
<font color=green>Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list ["King", "England"] was passed to our function, the function returned a DataFrame of 152 rows. Every row had the strings "King" and "England" somewhere in its " Question".<br> </font>
<font color=green>Note that in this example, we found 152 rows by filtering the entire dataset. You can download the entire dataset at the start or end of this project. The dataset used on Codecademy is only a fraction of the dataset so you won’t find as many rows.<br> </font>
<font color = green> Test your function by printing out the column containing the question of each row of the dataset. </font>



In [7]:
def questions_with_words(word_list):
    
    frame1 = jeopardy.copy(deep = True)
    
    makeitlower = lambda x: x.lower() 

    for word in word_list:
        word = word.lower()
        frame1['Also_Question'] = frame1.Question.apply(makeitlower)
        frame1['temp_column'] = frame1.Also_Question.str.contains(word, regex = True)
        frame1['Question'] = frame1.apply(lambda row: 'not_in_list' if row['temp_column'] == False \
                                               else row['Question'], axis = 1) 
        
    also_good_questions_list = frame1[frame1.Question != 'not_in_list']
                
    return also_good_questions_list

I'm including the Codecademy solution in the cell below; it will be clearly labeled as their code. In my solution, I made use of the Codecademy hint about using a lambda function. As it turns out, I needed the lambda function to make the strings in the "Questions" column lowercase (as .lower() couldn't be applied to an entire Series), and in my solution I was determined to use the Pandas function .str.contains().<br>
Here's some code that I got working before I got the above code (which uses str.contains()) to work. In all honesty, it's pretty much what the Codecademy solution does, just using a while loop instead of all().<br>

    def questions_with_words_too(word_list):
        frame = jeopardy.copy(deep = True)
     
        isitthere = lambda x: word.lower() in x.lower()
         
        i = 0
        while i < len(word_list):
            word = word_list[i]
            frame['temp_column'] = frame.Question.apply(isitthere)
            frame['Question'] = frame.apply(lambda row: 'blank' if row['temp_column'] == False else row['Question'],\
                                        axis = 1) 
            i = i+1
        
        good_questions_list = frame[frame.Question != 'blank']

        return good_questions_list

Note that the Codecademy solution takes much less time to run than mine does (either of mine actually), because my solution makes use of a "for" loop (or "while loop if you are looking at the code here in these comments) while Codecademy uses the built-in pandas function all(). That's a bit of coding inefficiency on my end that I hope will be minimized as I program more with Python and related packages. Basically, my code is the thing most similar in spirit to what I would've written if you'd asked me to solve this problem in IDL for an input structure or array, and was one of my favourite tactics when dealing with filtering problems in SDSS/SDSS-III BOSS CMASS data.

At any rage, below is the Codecademy solution.

In [8]:
def codecademy_questions_with_words(word_list):
    filter = lambda x: all(word.lower() in x.lower() for word in word_list)
    return jeopardy.loc[jeopardy["Question"].apply(filter)]

Now let's call my function (and the Codecademy one) on some lists of words I believe would reasonably be found together in a Jeopardy question. First I'll call the two functions for "King" and "England," to make sure that my solution yields the same answers as the Codecademy one, and then for some other lists of words just for fun. Also, I will print out the .info() for each DataFrame rather than the header, as the "Questions" Series gets kinda long.

In [9]:
questions_we_care_about = questions_with_words(['King','England'])
codecademy_solution = codecademy_questions_with_words(['King','England'])

print(questions_we_care_about.info())
print(codecademy_solution.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 4953 to 216789
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Show_Number    152 non-null    int64 
 1   Air_Date       152 non-null    object
 2   Round          152 non-null    object
 3   Category       152 non-null    object
 4   Value          152 non-null    object
 5   Question       152 non-null    object
 6   Answer         152 non-null    object
 7   Value_Num      152 non-null    int64 
 8   Year           152 non-null    int64 
 9   Month          152 non-null    int64 
 10  Day            152 non-null    int64 
 11  Also_Question  152 non-null    object
 12  temp_column    152 non-null    bool  
dtypes: bool(1), int64(5), object(7)
memory usage: 15.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 4953 to 216789
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       ---------

In light of the prompt below, I actually updated my function "questions_with_words" to have its call to .str.contains() default to regex = True rather than regex = False; being able to use regular expressions might help with the problem of returning quesions with one or more words having a substring of an input word, rather than the word itself. Let's see what the difference is if I use regex in my function call, and leave the Codecademy solution as-is:

In [10]:
questions_we_care_about_too = questions_with_words(['\sKing','England'])
codecademy_solution_too = codecademy_questions_with_words(['King','England'])

print(questions_we_care_about_too.info())
print(codecademy_solution_too.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 4953 to 216789
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Show_Number    130 non-null    int64 
 1   Air_Date       130 non-null    object
 2   Round          130 non-null    object
 3   Category       130 non-null    object
 4   Value          130 non-null    object
 5   Question       130 non-null    object
 6   Answer         130 non-null    object
 7   Value_Num      130 non-null    int64 
 8   Year           130 non-null    int64 
 9   Month          130 non-null    int64 
 10  Day            130 non-null    int64 
 11  Also_Question  130 non-null    object
 12  temp_column    130 non-null    bool  
dtypes: bool(1), int64(5), object(7)
memory usage: 13.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 4953 to 216789
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       ---------

Using regex with my function (which I updated to allow for regular expressions) yields fewer "hits" than the Codecademy function. I didn't investigate the whole DataFrame "questions_we_care_about_too", but at least the first 10 rows or so had questions which contained "k/King" and "England," and if there was something like "vikings" in the question it was because the former two words were in the question as well.

Let's move on to searching for things that don't involve King(s) of England:

In [11]:
#questions_we_care_about_2 = questions_with_words(['art','France'])
#codecademy_solution_2 = codecademy_questions_with_words(['art','France'])

# Just for fun, let's see how many times my alma mater (Colgate University) shows up:
#questions_we_care_about_2 = questions_with_words(['Colgate'])
#codecademy_solution_2 = codecademy_questions_with_words(['Colgate'])
# Though note that this will retrive questionss about Colgate-Palmolive as well, not just the University. 

# And let's see how many times my graduate alma mater (UPenn) shows up:
questions_we_care_about_2 = questions_with_words(['Pennsylvania','University'])
codecademy_solution_2 = codecademy_questions_with_words(['Pennsylvania','University'])
# Though this suffers from an even worse problem in that it pulls questions dealing with any University in 
# Pennslvania, including Penn State, Washington & Jefferson, and Lincoln.

print(questions_we_care_about_2.info())
print(codecademy_solution_2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 18081 to 191529
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Show_Number    6 non-null      int64 
 1   Air_Date       6 non-null      object
 2   Round          6 non-null      object
 3   Category       6 non-null      object
 4   Value          6 non-null      object
 5   Question       6 non-null      object
 6   Answer         6 non-null      object
 7   Value_Num      6 non-null      int64 
 8   Year           6 non-null      int64 
 9   Month          6 non-null      int64 
 10  Day            6 non-null      int64 
 11  Also_Question  6 non-null      object
 12  temp_column    6 non-null      bool  
dtypes: bool(1), int64(5), object(7)
memory usage: 630.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 18081 to 191529
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       -------

<font color=green>
Codecademy: <br>
</font>
<font color=green>
Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust. <br>
</font>
<font color=green>
For example, think about capitalization. We probably want to find questions that contain the word "King" or "king".
You may also want to check to make sure you don’t find rows that contain substrings of your given words. For example, our function found a question that didn’t contain the word "king", however it did contain the word "viking" — it found the "king" inside "viking". Note that this also comes with some drawbacks — you would no longer find questions that contained words like "England's".
</font>

Well, I already kinda took care of the capitalization thing with calls to .lower() for both the Question Series and the input list of strings. And in light of the above prompt, I went back to fix my original function for the "retrieve a substring" problem, and wrote about the results above. So here let's try and fix the problem I noted above with my "Colgate" search and let you search by question as well as category:

In [12]:
def questions_with_words_better(word_list, category_list):
    
    frame2 = jeopardy.copy(deep = True)
    
    makeitlower = lambda x: x.lower() 

    for word in word_list:
        word = word.lower()
        frame2['Also_Question'] = frame2.Question.apply(makeitlower)
        frame2['temp_column1'] = frame2.Also_Question.str.contains(word, regex = True)
        frame2['Question'] = frame2.apply(lambda row: 'not_in_list1' if row['temp_column1'] == False \
                                          else row['Question'], axis = 1) 
    for phrase in category_list:
        phrase = phrase.lower()
        frame2['Also_Category'] = frame2.Category.apply(makeitlower)
        frame2['temp_column2'] = frame2.Also_Category.str.contains(phrase, regex = True)
        frame2['Category'] = frame2.apply(lambda row: 'not_in_list2' if row['temp_column2'] == False \
                                        else row['Category'], axis = 1) 
        
    good_questions_list = frame2[(frame2.Question != 'not_in_list1') & (frame2.Category != 'not_in_list2') ]
                
    return good_questions_list

Naturally, since I re-set the default call to str.contains() above to include regex = True, I've done that again here. Once again, my function takes awhile to run, as it uses two "for" loops, but it runs and seems to give reasonable results for the limited number of question keywords/category keywords I have tried.

Well, now that we've got a "better"(?) question filtering function, let's try it on some lists of question keywords and category keywords that I think would be reasonably likely to be found together in a Jeopardy question:

In [13]:
# Kings of England are a history topic, right?
#questions_we_care_about_3 = questions_with_words_better(['\sKing','England'], ['history'])
#print(questions_we_care_about_3.info())

# Let's see if we can filter out Colgate-Palmolive from my Colgate University searches:
#questions_we_care_about_3 = questions_with_words_better(['Colgate'], ['college'])
#print(questions_we_care_about_3.Question)
# Yep! That got rid of the Colgate-Palmolive problem! But I think I lost a question dealing with that time Bill Cosby
# was the graduation speaker.

# Let's see if Jeopardy has ever asked about Finnish music; I have read in various places that Finland has the
# highest per capita number of heavy metal bands on Earth:
#questions_we_care_about_3 = questions_with_words_better(['Finland\s'], ['music'])
#print(questions_we_care_about_3.Question)
# Nope, doesn't look like it.

# And finally, since I have fond memories of Philly Beer Week, let's see what Jeopardy has asked about
# breweries in Pennsylvania:
#questions_we_care_about_3 = questions_with_words_better(['Pennsylvania'], ['beer'])
#print(questions_we_care_about_3.info())

# Hmm, doesn't look like they've asked about breweries in Pennsylvania. Let's try just beer in general:
questions_we_care_about_3 = questions_with_words_better(['beer'], ['\W'])
print(questions_we_care_about_3.info())
# Well, at least beer in general has been asked about 355 times!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1443 to 216329
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Show_Number    355 non-null    int64 
 1   Air_Date       355 non-null    object
 2   Round          355 non-null    object
 3   Category       355 non-null    object
 4   Value          355 non-null    object
 5   Question       355 non-null    object
 6   Answer         355 non-null    object
 7   Value_Num      355 non-null    int64 
 8   Year           355 non-null    int64 
 9   Month          355 non-null    int64 
 10  Day            355 non-null    int64 
 11  Also_Question  355 non-null    object
 12  temp_column1   355 non-null    bool  
 13  Also_Category  355 non-null    object
 14  temp_column2   355 non-null    bool  
dtypes: bool(2), int64(5), object(8)
memory usage: 39.5+ KB
None


<font color = green>
Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the “difficulty” of certain topics. For example, what is the average value of questions that contain the word "King"?
Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.
</font>

In [14]:
king_questions = questions_with_words_better(['King'], ['\W'])
average_king_value = king_questions.Value_Num.mean()
print(average_king_value)

771.8833850722094


Maybe Codecademy means the average value of questions that contain the words "King" and "England"?

In [15]:
average_kingengland_value = questions_we_care_about.Value_Num.mean()
print(average_kingengland_value)

886.8421052631579


<font color = green>
Codecademy: <br>
</font>
<font color = green>
Write a function that returns the count of the unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word "King", we could then find all of the unique answers to those questions. The answer “Henry VIII” appeared 3 times and was the most common answer.
</font>

I think what Codecademy is getting at with this prompt is that you should be able to get the number of unique answers to every question in whatever DataFrame you'd like, whether the full DataFrame or one filtered by either "questions_with_words" or "questions_with_words_better." To that end, it seems easiest to me to just pass in to the function a DataFrame which results from a call to "questions_with_words" or "questions_with_words_better," and use the function itself to just get the counts:

In [16]:
def unique_answer_count(df):
    all_counts = df.groupby(['Answer']).Question.nunique().reset_index()
    all_counts.rename(columns = {"Question":"Num. of Answers"}, inplace = True)
    all_counts.sort_values(by='Num. of Answers', ascending = False, inplace = True)
    return all_counts

At first, this seemed somewhat of a silly function to define, but if one wants to do this for numerous DataFrames, I suppose it makes sense to do rather than repeat the same line of code over and over again.

At any rate, let's test this function on the DataFrame we get by using Codecademy's function for "King," and see if we get what they got.

In [17]:
codecademy_king_dataframe = codecademy_questions_with_words(['King'])
all_codecademy_king_counts = unique_answer_count(codecademy_king_dataframe)
print(all_codecademy_king_counts)

                Answer  Num. of Answers
1543        Henry VIII               55
2963           Solomon               35
2723       Richard III               33
2070         Louis XIV               31
1027             David               30
...                ...              ...
1951     L. Frank Baum                1
1950           L'chaim                1
1949        Königsberg                1
1948  Kung Pao Chicken                1
5267           zombies                1

[5268 rows x 2 columns]


Hmmm. I didn't get 3; I got 55. Maybe they meant filtering the entire data set to only questions containing the words "King" and "England":

In [18]:
all_codecademy_king_counts = unique_answer_count(codecademy_solution_too)
print(all_codecademy_king_counts)

                      Answer  Num. of Answers
100    William the Conqueror                6
83   Richard the Lionhearted                3
28                    Edward                3
52                Henry VIII                3
93                    Wessex                3
..                       ...              ...
36                    George                1
35                    France                1
34                  Ethelred                1
33                    Engels                1
113       the Stone of Scone                1

[114 rows x 2 columns]


Well it doesn't seem to be the most *common* answer, but yeah, Henry VIII was an answer 3 times.

OK, so now to try it with my function on not only "King" and "England", but maybe some other word/word combinations too.

In [19]:
my_king_counts = unique_answer_count(questions_we_care_about_too)
print(my_king_counts)

                     Answer  Num. of Answers
84    William the Conqueror                5
46                  James I                3
43               Henry VIII                3
70  Richard the Lionhearted                3
30                 George I                3
..                      ...              ...
32   George III & Louis XVI                1
29   George Frideric Handel                1
28          George (I - IV)                1
27                   George                1
92       the Stone of Scone                1

[93 rows x 2 columns]


Well, I get the same Henry VIII counts as Codecademy does, so I suppose that is good.

Now let's try a fun one:

In [20]:
my_name_questions = questions_with_words_better(['Michelle'], ['\W'])
my_name_counts = unique_answer_count(my_name_questions)
print(my_name_counts)

                     Answer  Num. of Answers
12                    Chile                3
22               Full House                2
61  The Witches of Eastwick                2
41                    Obama                2
33               Lee Marvin                2
..                      ...              ...
32                Ladyhawke                1
34               Love Field                1
35       Married to the Mob                1
1         "Dangerous Minds"                1
71      the inaugural balls                1

[72 rows x 2 columns]


My first name doesn't come up very often in a Jeopardy question, but when it does the most common answer is "Chile."

<font color = green>
Codecademy: <br>
</font>
<font color = green>
    Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven’t even started looking at yet. Here are some ideas on ways to continue working with this data:<br>
</font>
<font color = green>
    Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word "Computer" compared to questions from the 2000s?<br>
</font>
<font color = green>
	Is there a connection between the round and the category? Are you more likely to find certain categories, like "Literature" in Single Jeopardy or Double Jeopardy?
</font>

I picked these two prompts because they seemed the most interesting to me. Let's tackle them first, then get to a couple of questions I thought up.<br>

First up: How do the questions change over time? <br>

I think it's best to actually modify my questions_with_words_better function to carry out these investigations, as one can filter by Category in addition to Question if one wants. (Or if one doesn't, just pass the regex '\W' in as a list.) Since I made separate columns for year, month, and day, but you may not want to filter by all/any of these, I'll set year_list, month_list, and day_list to have default values of [1984,2012], [01,12], and [01,31] respectively. <br>
At any rate, let's go ahead and carry out that modification:

In [21]:
def questions_with_words_best(word_list, category_list, year_list = [1984,2012], \
                              month_list = [0,12], day_list = [0,31]):
    
    frame3 = jeopardy.copy(deep = True)
    
    makeitlower = lambda x: x.lower() 

    for word in word_list:
        word = word.lower()
        frame3['Also_Question'] = frame3.Question.apply(makeitlower)
        frame3['temp_column1'] = frame3.Also_Question.str.contains(word, regex = True)
        frame3['Question'] = frame3.apply(lambda row: 'not_in_list1' if row['temp_column1'] == False \
                                          else row['Question'], axis = 1) 
    for phrase in category_list:
        phrase = phrase.lower()
        frame3['Also_Category'] = frame3.Category.apply(makeitlower)
        frame3['temp_column2'] = frame3.Also_Category.str.contains(phrase, regex = True)
        frame3['Category'] = frame3.apply(lambda row: 'not_in_list2' if row['temp_column2'] == False \
                                        else row['Category'], axis = 1) 
        
    questions_list_step1 = frame3[(frame3.Question != 'not_in_list1') & (frame3.Category != 'not_in_list2') ]
    
    questions_list_step2 = questions_list_step1[(questions_list_step1.Year >= year_list[0]) &
                                                (questions_list_step1.Year <= year_list[1])]
    
    questions_list_step3 = questions_list_step2[(questions_list_step2.Month >= month_list[0]) &
                                                (questions_list_step2.Month <= month_list[1])]
        
    good_questions_list = questions_list_step3[(questions_list_step3.Day >= day_list[0]) &
                                                (questions_list_step3.Day <= day_list[1])]
    
    return good_questions_list

# Note that there's a limitation on using this function to filter by date: if you want to filter by something more 
# specific than a range of years or months -- like say, you wanted January 2000 through January 2010 -- my 
# function as currently written isn't going to do that very well. If you want to do that, it might actually be easier
# to select those specific dates from the main DataFrame and then pass that DataFrame to questions_with_words_better.

Alright. Let's test this out on some stuff we already know the answer to:

In [22]:
new_function_test = questions_with_words_best(['King','England'], ['\W'])
print(new_function_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 4953 to 216789
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Show_Number    152 non-null    int64 
 1   Air_Date       152 non-null    object
 2   Round          152 non-null    object
 3   Category       152 non-null    object
 4   Value          152 non-null    object
 5   Question       152 non-null    object
 6   Answer         152 non-null    object
 7   Value_Num      152 non-null    int64 
 8   Year           152 non-null    int64 
 9   Month          152 non-null    int64 
 10  Day            152 non-null    int64 
 11  Also_Question  152 non-null    object
 12  temp_column1   152 non-null    bool  
 13  Also_Category  152 non-null    object
 14  temp_column2   152 non-null    bool  
dtypes: bool(2), int64(5), object(8)
memory usage: 16.9+ KB
None


OK! That gets us back the same DataFrame we got before (i.e., questions_we_care_about). So now let's look at what Codecademy recommended: How many questions from the 90s use the word "Computer" compared to questions from the 2000s?<br>
I'll just use len() to get the number of questions rather than print out the full DataFrame info().

In [23]:
computers_90s = questions_with_words_best(['computer'], ['\W'], year_list = [1990,1999])
computers_00s = questions_with_words_best(['computer'], ['\W'], year_list = [2000,2009])

print(len(computers_90s), len(computers_00s))

98 268


Just as one might have suspected, questions about computers became more common as they became a more common fixture of everyday life. To that end, how about cell phones?

In [24]:
cellphone_90s = questions_with_words_best(['cell phone'], ['\W'], year_list = [1990,1999])
cellphone_00s = questions_with_words_best(['cell phone'], ['\W'], year_list = [2000,2009])

print(len(cellphone_90s), len(cellphone_00s))

2 24


Yep, questions about cell phones becamse more common, too. One could track DE-popularity as well, like say how rare questions about the USSR became after 1995, or what category they were found in, etc. Or one could see if certain Questions or Categories were more likely to pop up in spring, or summer, or in the first half of the month, etc.

Now let's investigate connections between the round and the category. First, let's define a function to see how many times Questions of a particular Category appeared in each Round. The DataFrame passed to the function category_round_count is assumed to have already been filtered by Category.

In [25]:
def category_round_count(df):
    all_counts = df.groupby(['Round']).Question.count().reset_index()
    all_counts.rename(columns = {"Question":"Num. of Appearances"}, inplace = True)
    all_counts.sort_values(by='Num. of Appearances', ascending = False, inplace = True)
    return all_counts

Now let's test!

In [26]:
literacy = questions_with_words_best(['\W'], ['Literature'])
literacy_counts = category_round_count(literacy)
print(literacy_counts)

              Round  Num. of Appearances
0  Double Jeopardy!                 1054
2         Jeopardy!                  423
1   Final Jeopardy!                   82


In [27]:
sportsball = questions_with_words_best(['\W'], ['Sports'])
sportsball_counts = category_round_count(sportsball)
print(sportsball_counts)

              Round  Num. of Appearances
2         Jeopardy!                 1139
0  Double Jeopardy!                  845
1   Final Jeopardy!                   60


Sports is a common Jeopardy! category, whereas Literature is a common Double Jeopardy! category.

Now what questions do *I* have independently of Codecademy prompts? <br>

Well, let's do a basic one first: What's the average Value for a particular Category?

In [28]:
print('Average value of Questions in Literature-based categories:', literacy.Value_Num.mean())
print('Average value of Questions in Sports-based categories:', sportsball.Value_Num.mean())

Average value of Questions in Literature-based categories: 707.8896728672225
Average value of Questions in Sports-based categories: 665.6555772994129


It makes sense that Literature-based questions should have a higher average value than Sports-based questions, because Sports questions are more likely to appear in Jeopardy! rounds, and each question is worth half in Jeopardy! what it would be in Double Jeopardy!. One could compile numbers for different Categories if one wanted.

Speaking of Categories. Are certain Categories more likely to appear in a particular Month? (I'd expect a category containing "holidays" to be kinda popular in December, for example.) Let's find out. Assume a DataFrame which has already been filtered by category:

In [29]:
def category_by_month(df):
    catbymonth = df.groupby(['Month']).Question.nunique().reset_index()
    catbymonth.rename(columns = {"Question":"Num. of Appearances"}, inplace = True)
    catbymonth.sort_values(by='Num. of Appearances', ascending = False, inplace = True)
    return catbymonth

In [30]:
holiday_questions = questions_with_words_best(['\W'], ['holiday'])
holiday_by_month = category_by_month(holiday_questions)
print(holiday_by_month)

    Month  Num. of Appearances
9      11                   61
4       5                   53
1       2                   49
0       1                   47
10     12                   47
3       4                   45
8      10                   39
6       7                   25
5       6                   22
2       3                   15
7       9                   14


Huh. "Holiday" is most common in November! Given that Halloween just ended, Thanksgiving is in that month, and the Christmas shopping season is on the horizon, I guess that makes sense in retrospect. And, no "Holiday" questions in August? Weird! But upon further investigation, there doesn't seem to be any data at all in the data base from August of any year! I wonder why that is.

Well, anyway, let's look at some other categories too:

In [31]:
literacy_by_month = category_by_month(literacy)
print(literacy_by_month)

sportsball_by_month = category_by_month(sportsball)
print(sportsball_by_month)

    Month  Num. of Appearances
9      11                  229
2       3                  165
4       5                  159
10     12                  151
1       2                  142
3       4                  138
0       1                  132
5       6                  120
8      10                  118
7       9                  111
6       7                   94
    Month  Num. of Appearances
9      11                  239
8      10                  233
2       3                  221
10     12                  201
4       5                  193
5       6                  182
6       7                  179
0       1                  178
3       4                  161
1       2                  145
7       9                  112


Broadly speaking, you're more likely to encounter a Sports-like question in winter than in summer. Also, Literature-like questions are very popular in November and not at all common in July.

Hmmm. Let's see what the most common Final Jeopardy! Categories are:

In [32]:
final_jeopardy = jeopardy[jeopardy.Round == 'Final Jeopardy!']

final_jeopardy_category_counts = final_jeopardy.groupby(['Category']).Question.nunique().reset_index()
final_jeopardy_category_counts.rename(columns = {"Question":"Num. of Appearances"}, inplace = True)
final_jeopardy_category_counts.sort_values(by='Num. of Appearances', ascending = False, inplace = True)

print(final_jeopardy_category_counts)

             Category  Num. of Appearances
1838  U.S. PRESIDENTS                   50
1898     WORD ORIGINS                   34
1560   STATE CAPITALS                   31
332           AUTHORS                   27
1927    WORLD LEADERS                   26
...               ...                  ...
742    FATHERS & SONS                    1
739   FAMOUS WEDDINGS                    1
738    FAMOUS VOYAGES                    1
737      FAMOUS TEXTS                    1
1951          ZOOLOGY                    1

[1952 rows x 2 columns]


Looks like U.S. Presidents is a somewhat common Final Jeopardy! Category, as are Word Origins and State Capitals.

Let's get the median Value of Questions by round (assuming that you have already filtered the data set by Question, using questions_with_words_best I hope!): 

In [33]:
def get_median_value(df):
    get_stat = df.groupby(['Round']).Value_Num.median().reset_index()
    get_stat.sort_values(by='Value_Num', ascending = False, inplace = True)
    return get_stat

Now run:

In [34]:
# First, look at Questions containing the words "King" and "England""
king_median= get_median_value(questions_we_care_about)
print('Questions with "King" and "England":')
print(king_median)
# Now look at Questions containing my first name:
name_median = get_median_value(my_name_questions)
print('Questions containing "Michelle":')
print(name_median)

# I originally input DataFrames filtered by Category and fed them to get_median_values, but I decided that this was a 
# dumb idea as I was just going to get the "middle of the road" question values every time.

Questions with "King" and "England":
              Round  Value_Num
0  Double Jeopardy!       1000
2         Jeopardy!        500
1   Final Jeopardy!          0
Questions containing "Michelle":
              Round  Value_Num
0  Double Jeopardy!        800
1         Jeopardy!        400


If question Value is a broad indicator of difficulty, questions containing "King" and "England" tend to be harder than questions containing "Michelle."